In [1]:
!nvidia-smi

Mon Aug 22 10:15:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 30%   33C    P8    20W / 320W |   7424MiB / 10240MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch

In [1]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm
import re, magic

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage import zoom
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix

#vit
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [2]:
model =timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

model

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
   

In [3]:
train=pd.read_excel("../1205_stool_train.xlsx")

In [4]:
train

,image_id,label
0,100_0042_0.jpg,0
1,100_0044_0.jpg,0
2,100_0045_0.jpg,0
3,100_0048_0.jpg,0
4,100_0049_0.jpg,0
...,...,...
36046,V1449_p2_SSL_old_0042_0.jpg,0
36047,V1449_p2_SSL_old_0043_0.jpg,0
36048,V1449_p2_SSL_old_0044_0.jpg,0
36049,V1449_p2_SSL_old_0045_0.jpg,0


In [5]:
train.label.value_counts()

0    33194
1     2857
Name: label, dtype: int64

In [8]:
train

,image_id,label
0,100_0042_0.jpg,0
1,100_0044_0.jpg,0
2,100_0045_0.jpg,0
3,100_0048_0.jpg,0
4,100_0049_0.jpg,0
...,...,...
36046,V1449_p2_SSL_old_0042_0.jpg,0
36047,V1449_p2_SSL_old_0043_0.jpg,0
36048,V1449_p2_SSL_old_0044_0.jpg,0
36049,V1449_p2_SSL_old_0045_0.jpg,0


In [6]:
valid=pd.read_excel("../1205_stool_valid.xlsx")

In [7]:
valid.label.value_counts()

0    8286
1     646
Name: label, dtype: int64

In [7]:
tst_dir = glob('D:/@image_data/open_data/@sun_data/*.jpg')
tst_df = pd.DataFrame(tst_dir, columns=['image_id'])
tst_df['image_id'] = tst_df['image_id'].apply(lambda x: os.path.basename(x))
tst_df['label'] = tst_df['image_id'].apply(lambda x: 1 if 'stool' in x else 0)
tst_df
test=tst_df

In [8]:
train.label.value_counts()

0    33194
1     2857
Name: label, dtype: int64

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

In [10]:
class ColonDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

In [11]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize, SmallestMaxSize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            #SmallestMaxSize(CFG['img_size']),
            RandomResizedCrop(CFG['img_size'], CFG['img_size'], p=0.5),
            Resize(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)


def get_inference_transforms():
    return Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [12]:
class ColonImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class=2, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained, num_classes=n_class)
        try :
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, n_class)

        except :
            
            try : 
                n_features = self.model.fc.in_features
                self.model.fc = nn.Linear(n_features, n_class)
            
            except :
                try:
                    n_features = self.model.head.fc.in_channels
                    self.model.head.fc = nn.Conv2d(n_features,n_class,kernel_size=(1, 1), stride=(1, 1))
            
                except:
                    n_features = self.model.head.in_features
                    self.model.head = nn.Linear(n_features, n_class)
            
    def forward(self, x):
        x = self.model(x)
        return x

In [13]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [14]:
CFG = {
    'fold_num': 5,
    'seed': 25,
    'model_arch': 'efficientnet_b0',
    'img_size': 128,
    'epochs': 100,
    'train_bs': 10,
    'valid_bs': 10,
    'lr': 1e-5,
    'num_workers': 0,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 1,
    'used_epochs': [74],
    'weights': [1]
}

In [15]:
test

,image_id,label
0,case100_case_M_20181106093315_0U62372110682814...,0
1,case100_case_M_20181106093315_0U62372110682814...,0
2,case100_case_M_20181106093315_0U62372110682814...,0
3,case100_case_M_20181106093315_0U62372110682814...,0
4,case100_case_M_20181106093315_0U62372110682814...,0
...,...,...
49387,stool_v1_4_sun2_00042196_1.jpg,1
49388,stool_v1_4_sun2_00042197_1.jpg,1
49389,stool_v1_4_sun2_00042199_1.jpg,1
49390,stool_v1_4_sun2_00042202_1.jpg,1


In [16]:
train_dir ='D:/polyps'
valid_dir = 'D:/polyps'
test_dir ='D:/@image_data/open_data/@sun_data/'
model_dir = '../../models/221211/efficientnet_b0/stool_classfication_124_{}_fold_0_{}'.format(CFG['model_arch'], CFG['used_epochs'][0])

if __name__ == '__main__':
     # for training only, need nightly build pytorch

    seed_everything(CFG['seed'])
    
    folds = StratifiedKFold(n_splits=CFG['fold_num'], shuffle=True, random_state=CFG['seed']).split(np.arange(train.shape[0]), train.label.values)
    
    for fold, (trn_idx, val_idx) in enumerate(folds):
        
        print('Inference fold {} started'.format(fold))

        
        #train_ = train.loc[trn_idx,:].reset_index(drop=True)
        train_=train
        train_ds = ColonDataset(train_, train_dir, transforms=get_valid_transforms(), output_label=False)
        
        #valid_ = train.loc[val_idx,:].reset_index(drop=True)
        valid_=valid
        valid_ds = ColonDataset(valid_, valid_dir, transforms=get_valid_transforms(), output_label=False)
        
        test = test
        test_ds = ColonDataset(test, test_dir, transforms=get_valid_transforms(), output_label=False)
        
        trn_loader = torch.utils.data.DataLoader(
            train_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )
        
        val_loader = torch.utils.data.DataLoader(
            valid_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )
        
        tst_loader = torch.utils.data.DataLoader(
            test_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )

        device = torch.device(CFG['device'])
        model = ColonImgClassifier(CFG['model_arch'], train.label.nunique()).to(device)
        
        trn_preds=[]
        val_preds = []
        tst_preds = []
        
        #for epoch in range(CFG['epochs']-3):
        for i, epoch in enumerate(CFG['used_epochs']):    
            model.load_state_dict(torch.load(model_dir))
            
            with torch.no_grad():
                for _ in range(CFG['tta']):
                    trn_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, trn_loader, device)]
                    val_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, val_loader, device)]
                    tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, tst_loader, device)]
                    
                    
        trn_preds= np.mean(trn_preds, axis=0) 
        val_preds = np.mean(val_preds, axis=0) 
        tst_preds = np.mean(tst_preds, axis=0) 
        
        print('fold {} validation loss = {:.5f}'.format(fold, log_loss(valid_.label.values, val_preds)))
        print('fold {} validation accuracy = {:.5f}'.format(fold, (valid_.label.values==np.argmax(val_preds, axis=1)).mean()))
        
        del model
        torch.cuda.empty_cache()
        
        # we'll train fold 0 first
        if fold == 0:
            break 

Inference fold 0 started


100%|██████████████████████████████████████████████████████████████████████████████| 4940/4940 [11:31<00:00,  7.14it/s]

fold 0 validation loss = 0.03054
fold 0 validation accuracy = 0.99205


In [18]:
# test
# test['label'] = test['image_id'].apply(lambda x : 0 if 'cancer' in x else 1)
train['pred'] = np.argmax(trn_preds, axis=1)
train['confidence score'] =np.max(trn_preds, axis=1)
train_acc = np.sum(train.label == train.pred) / len(train)
train_matrix = confusion_matrix(train['label'], train['pred'])
print(round(train_acc, 5))
print(train_matrix)

0.99947
[[33186     8]
 [   11  2846]]


In [27]:

valid['pred'] = np.argmax(val_preds, axis=1)
valid['confidence score'] =np.max(val_preds, axis=1)
valid_acc = np.sum(valid.label == valid.pred) / len(valid)
valid_matrix = confusion_matrix(valid['label'], valid['pred'])
print(round(valid_acc, 5))
print(valid_matrix)

0.99205
[[8275   11]
 [  60  586]]


In [28]:

test['pred'] = np.argmax(tst_preds, axis=1)
test['confidence score'] =np.max(tst_preds, axis=1)
test_acc = np.sum(test.label == test.pred) / len(test)
test_matrix = confusion_matrix(test['label'], test['pred'])
print(round(test_acc, 5))
print(test_matrix)

0.98558
[[48470   663]
 [   49   210]]


In [24]:
test

,image_id,label,pred,confidence score
0,case100_case_M_20181106093315_0U62372110682814...,0,0,0.999997
1,case100_case_M_20181106093315_0U62372110682814...,0,0,0.999928
2,case100_case_M_20181106093315_0U62372110682814...,0,0,0.999992
3,case100_case_M_20181106093315_0U62372110682814...,0,0,0.999988
4,case100_case_M_20181106093315_0U62372110682814...,0,0,0.999695
...,...,...,...,...
49387,stool_v1_4_sun2_00042196_1.jpg,1,1,0.762572
49388,stool_v1_4_sun2_00042197_1.jpg,1,1,0.978017
49389,stool_v1_4_sun2_00042199_1.jpg,1,0,0.988263
49390,stool_v1_4_sun2_00042202_1.jpg,1,0,0.572435


In [25]:
def score(df):
    TP=len(df.query("label==1 and pred==1 "))
    TN=len(df.query("label==0 and pred==0 "))
    FP=len(df.query("label==0 and pred==1 "))
    FN=len(df.query("label==1 and pred==0 "))
    Sen=round(TP/(TP+FN),4)
    Spe=round(TN/(TN+FP),4) if TN+FP != 0 else 0
    PPV=round(TP/(TP+FP),4)
    NPV=round(TN/(TN+FN),4) if TN+FN != 0 else 0
    ACC=round((TP+TN)/(TP+TN+FP+FN),4)
    print("TP:",TP)
    print("FP:",FP)
    print("TN:",TN)
    print("FN:",FN)
    print("Sen:",Sen)
    print("Spe:",Spe)
    print("PPV:",PPV)
    print("NPV:",NPV)
    print("ACC:",ACC)

In [26]:
score(test)

TP: 210
FP: 663
TN: 48470
FN: 49
Sen: 0.8108
Spe: 0.9865
PPV: 0.2405
NPV: 0.999
ACC: 0.9856
